In [ ]:
from tqdm import tqdm
import pandas as pd
import pickle

In [2]:

graph_dict = {}

with open('/home/harsh.d/shruti/ALL_ldb_graph_specter.pkl', 'rb') as f:
    while True:
        try:
            local_graph_dict = pickle.load(f)
            graph_dict.update(local_graph_dict)
        except Exception as ex:
            print(ex)
            break
print(len(graph_dict))

Ran out of input
7744


In [7]:
with open('../Benchmark Data/leaderboard_details', 'rb') as config_dictionary_file:
    r = pickle.load(config_dictionary_file)  

In [5]:
with open('../Benchmark Data/intersection_dataset_benchmark_paperID', 'rb') as config_dictionary_file:
   intersection_dataset_benchmark_paperID = pickle.load(config_dictionary_file)  

In [6]:
uu = pd.read_pickle('../Benchmark Data/Table_Citations.pkl')

In [18]:
def give_stats(list_name):
    print("Mean : ",statistics.mean(list_name))
    print("Median : ",statistics.median(list_name))
    print("Mode : ",statistics.mode(list_name))
    print("Standerd Deviation : ",statistics.stdev(list_name))
    print("Maximum Value : ", max(list_name))
    print("Minimum Value : ",min(list_name))
    print("Count of 100 : ", list_name.count(100))
    print("Count of 0 : ", list_name.count(0))
    print("Total Length : ", len(list_name))

In [33]:
from scipy.stats import spearmanr
def calculate_spearman_rank_correlation(actual_ranks, page_ranks):
    # use the PageRank scores as ranks
    ranks = page_ranks
    # calculate Spearman's rank correlation coefficient
    corr, p_value = spearmanr(actual_ranks, ranks)
    return corr, p_value

In [32]:
# Initialize variables
number_of_corrected_ranks = 0
zero_rank_leaderboard = 0
not_in_sorted_order = 0
number_task = 0
r_list = []

# Loop through each dataset and task in the graph_dict
for dataset in tqdm(graph_dict):
    for task in graph_dict[dataset]:
        # Get the list of tuples for this dataset and task
        my_list_of_tuples = graph_dict[dataset][task]
        
        # If the list of tuples is not empty
        if len(my_list_of_tuples)>0:
            # Get the list of arxiv ids for this dataset and task, ignoring certain ids
            tt = [x for x in r[dataset][task]['arxiv id'] if x != 0 and not x.startswith(('2205', '2206', '2207', '2208', '2209', '2210', '2211', '2212', '23'))]
            
            # Combine the arxiv ids from both source and destination nodes in the list of tuples
            arxiv_id = []
            arxiv_id.extend([x[0] for x in my_list_of_tuples])
            arxiv_id.extend([x[1] for x in my_list_of_tuples])
            arxiv_id = set(arxiv_id)
            
            # Calculate the overlap between the arxiv ids in the list of tuples and the arxiv ids in the ground truth
            r_list.append(len(arxiv_id.intersection(set(tt)))/len(arxiv_id)*100)        
            
            # Create a directed graph using networkx library
            G = nx.DiGraph()
            # Add nodes and edges to the graph
            for source, destination, weight in my_list_of_tuples:
                G.add_edge(source, destination, weight=weight)
            
            # Calculate the PageRank score for each node in the graph
            pr = nx.pagerank(G, weight='weight')
            
            # Get the PageRank scores for the arxiv ids in the ground truth and check if they are sorted in descending order
            ranks = []
            for k in arxiv_id.intersection(set(tt)):
                ranks.append(pr[k])
            if len(ranks)>0 and ranks == sorted(ranks,reverse=True):
                number_of_corrected_ranks+=1
            elif len(ranks)==0:
                zero_rank_leaderboard+=1
            else:
                not_in_sorted_order +=1
        
        # Increment the number of tasks
        number_task+=1


100%|███████████████████████████████████████| 7744/7744 [02:27<00:00, 52.45it/s]


In [33]:
print('number_of_corrected_ranks:', number_of_corrected_ranks)
print('zero_rank_leaderboard:',zero_rank_leaderboard)
print('not_in_sorted_order:', not_in_sorted_order)
print('number_task:',number_task)

number_of_corrected_ranks: 1301
zero_rank_leaderboard: 3542
not_in_sorted_order: 840
number_task: 5827
